# 240. CIFAR-10 을 이용한 CNN 구축

- **CNN**을 학습하여 CIFAR-10 데이터베이스의 이미지를 분류합니다.

<img src='https://production-media.paperswithcode.com/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg' width=600 />


- mean, std ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 로 normalize 된 image 의 unnormalization 방법
    - image = image * 0.5 + 0.5

## Data Download 및 Data Loader 를 이용하여 Train, Validation data 준비

transforms.ToTensor()의 주요 특징:

1) 데이터 타입 변환: PIL 이미지나 NumPy ndarray를 torch.FloatTensor로 변환  
2) 스케일링: 이미지의 픽셀 값 범위를 [0, 255]에서 [0.0, 1.0]으로 스케일링  
3) 차원 재배열: PyTorch에서는 이미지 데이터를 [C, H, W] 형식(채널, 높이, 너비)으로 처리하므로 입력 이미지 데이터의 차원을 이 형식으로 자동 재배열

In [ ]:
# 이미지 데이터에 대한 변환(transform) 조합을 정의
# 이 변환은 이미지 데이터를 증강(augmentation)하고 전처리하는 과정을 포함합니다.
# CIFAR10 훈련 데이터셋을 다운로드하고, 위에서 정의한 변환을 적용
# CIFAR10 테스트 데이터셋을 다운로드하고, 위에서 정의한 변환을 적용
# DataLoader를 사용하여 훈련 데이터셋과 테스트 데이터셋을 배치로 로드합니다.
# shuffle=True는 훈련 데이터 로딩 시 데이터를 무작위로 섞어 오버피팅을 방지하는 데 도움이 됩니다.
# 테스트 데이터 로더에서는 데이터를 섞지 않습니다(shuffle=False).

### 일부 Data 시각화

- matplotlib 은 channel 위치가 last 이므로 transpose(1, 2, 0) 로 image 수정

In [ ]:
    # np.transpose 함수를 사용하여 'x' 텐서의 차원을 재배열
    # (1, 2, 0)은 채널 차원을 마지막으로 이동시키고, 높이와 너비 차원을 앞으로 가져옵니다.
    # 이 형식은 matplotlib와 같은 일부 라이브러리에서 이미지를 표시할 때 요구되는 형식입니다.
    # * 0.5 + 0.5 연산은 정규화를 되돌려 픽셀 값을 원본 범위(0~1)로 unnormalize

## Model build

### Custom Model

- Output Size = (W - F + 2P) / S + 1  
- output_size / Maxpool(2)

In [ ]:
def output_size(W, F, P, S, poolsize=1):

input image size : (32, 32)
kernel size : 3
padding : 1
stride : 1

In [ ]:
class Net(nn.Module):
    def __init__(self):
        # 3개의 입력 채널을 받아 16개의 출력 채널을 생성
        # 커널 크기는 3, 패딩은 1로 설정하여 입력 이미지의 크기를 유지합니다.
        # 16개의 입력 채널을 받아 32개의 출력 채널을 생성
        # 32개의 입력 채널을 받아 64개의 출력 채널을 생성
        # 첫 번째 완전 연결 층을 정의합니다. 입력 특징의 수는 4*4*64, 출력 특징의 수는 256입니다.
        # 두 번째 완전 연결 층을 정의합니다. 10개의 출력 클래스에 대응합니다.
    def forward(self, x):
        # 합성곱 층과 배치 정규화, ReLU 활성화 함수, 최대 풀링을 차례로 적용
        # 특징 맵을 1차원으로 평탄화
        # 드롭아웃 적용
        # 완전 연결 층과 배치 정규화, ReLU 활성화 함수 적용
        # 최종 출력 계산

`model.forward(train_data[0][0].unsqueeze(0).to(device))`

### nn.Sequential 이용


- ``torch.nn`` 에는 코드를 간단히 사용할 수 있는 또 다른 편리한 클래스인
[Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) 이 있습니다.  

- ``Sequential`` 객체는 그 안에 포함된 각 모듈을 순차적으로 실행합니다. 이것은 신경망을 작성하는 더 간단한 방법입니다.


In [ ]:
# nn.Sequential을 사용하여 순차적 모델을 정의합니다. 이 방식은 각 레이어를 순서대로 쌓아 올립니다.
    #  3개의 입력 채널(예: RGB 이미지)을 받아 16개의 출력 채널 생성
    # 커널 크기는 3, 스트라이드는 1, 패딩은 1로 설정
    # 16개의 채널에 대해 배치 정규화 수행
    # 활성화 함수로 ReLU 사용
    # 맥스 풀링 레이어: 커널 크기와 스트라이드를 2로 설정
    # 6개의 입력 채널을 받아 32개의 출력 채널 생성
    # 32개의 채널에 대해 배치 정규화 수행
    # 32개의 입력 채널을 받아 64개의 출력 채널 생성
    # 64개의 채널에 대해 배치 정규화 수행
    # 텐서를 1차원으로 평탄화
    # 드롭아웃을 적용하여 과적합 방지
    # 평탄화된 텐서를 받아 256개의 뉴런을 가진 레이어로 연결
    # 완전 연결 레이어의 배치 정규화
    # 10개의 출력을 가진 레이어로 클래스 분류

### Model Summary

In [ ]:
# 두 모델의 파라미터 수가 동일함을 확인

### Loss Function

In [ ]:
    # 에폭별 훈련 및 검증 결과 출력

### Model 평가

### model 이 어떤 image 들을 잘 맞추고 혹은 틀렸는지 시각화

In [ ]:
# 테스트 데이터셋을 배치 단위로 순회
# 예측값과 실제 레이블이 일치하는 경우의 인덱스를 찾음
# 예측값과 실제 레이블이 불일치하는 경우의 인덱스를 찾음
# 정확도 계산: 정확한 예측의 수를 전체 예측의 수로 나눈 후 100을 곱하여 백분율로 변환

In [ ]:
# 정확한 예측의 인덱스에서 처음 9개를 순회합니다.
 # 잘못된 예측의 인덱스에서 처음 9개를 순회합니다.

## Saving and loading the model

### Save and load the model via state_dict

In [ ]:
# 'Net' 클래스로부터 모델 인스턴스 생성
# 'load_state_dict()' 메서드를 사용하여 로드한 가중치를 현재 모델 인스턴스에 적용합니다.
# 'eval()' 메서드는 모델을 평가/테스트 모드로 설정하여, 드롭아웃, 배치 정규화 등이
# 학습 모드와는 다르게 동작하도록 합니다.

### Save and load entire model

In [ ]:
# Save